In [4]:
!pip install dash
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

# Cargar los datos desde el archivo CSV (ruta ajustada para el código local)
df = pd.read_csv('SeoulBikeData_utf8.csv')

# Transformaciones de los datos (basado en tu notebook)
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Holiday'] = df['Holiday'].map({'No Holiday': 0, 'Holiday': 1})
df['Functioning Day'] = df['Functioning Day'].map({'Yes': 1, 'No': 0})
df = pd.get_dummies(df, columns=['Seasons'], drop_first=True)

# Simular el modelo de regresión lineal múltiple
X = df[['Hour', 'Temperature(C)', 'Humidity(%)', 'Wind speed (m/s)', 'Holiday', 'Functioning Day']]
y = df['Rented Bike Count']
model = LinearRegression()
model.fit(X, y)

# Inicializar la aplicación Dash
app = dash.Dash(__name__)

# Layout del tablero
app.layout = html.Div(children=[
    html.H1(children='Tablero de Bicicletas Compartidas'),

    html.Div([
        html.Label('Seleccione la Hora del Día:'),
        dcc.Slider(id='input_hour', min=0, max=23, step=1, value=12,
                   marks={i: f'{i}:00' for i in range(0, 24)}),
        
        html.Label('Seleccione la Temperatura (°C):'),
        dcc.Slider(id='input_temp', min=-10, max=35, step=1, value=20,
                   marks={i: f'{i}°C' for i in range(-10, 36, 5)}),
        
        html.Label('Seleccione la Humedad (%):'),
        dcc.Slider(id='input_humidity', min=0, max=100, step=1, value=50,
                   marks={i: f'{i}%' for i in range(0, 101, 10)}),
        
        html.Label('Velocidad del viento (m/s):'),
        dcc.Slider(id='input_wind', min=0, max=10, step=0.5, value=5,
                   marks={i: f'{i} m/s' for i in range(0, 11)}),
        
        html.Label('¿Es un día festivo?:'),
        dcc.Dropdown(id='input_holiday', options=[{'label': 'Sí', 'value': 1}, {'label': 'No', 'value': 0}],
                     value=0),
        
        html.Label('¿Es un día hábil?:'),
        dcc.Dropdown(id='input_functioning_day', options=[{'label': 'Sí', 'value': 1}, {'label': 'No', 'value': 0}],
                     value=1)
    ], style={'padding': 20, 'width': '50%'}),

    html.H2('Predicción de bicicletas alquiladas:'),
    html.Div(id='output_prediction'),

    dcc.Graph(id='output_graph')
])

# Callback para actualizar la predicción y gráfico
@app.callback(
    [Output('output_prediction', 'children'),
     Output('output_graph', 'figure')],
    [Input('input_hour', 'value'),
     Input('input_temp', 'value'),
     Input('input_humidity', 'value'),
     Input('input_wind', 'value'),
     Input('input_holiday', 'value'),
     Input('input_functioning_day', 'value')]
)
def update_prediction(hour, temp, humidity, wind, holiday, functioning_day):
    # Realizar la predicción basada en los inputs
    X_new = np.array([[hour, temp, humidity, wind, holiday, functioning_day]])
    prediction = model.predict(X_new)[0]

    # Crear gráfico de barras para visualización de la demanda por hora
    filtered_df = df[(df['Temperature(C)'] == temp) & (df['Humidity(%)'] == humidity)]
    fig = px.bar(filtered_df, x='Hour', y='Rented Bike Count', title='Bicicletas alquiladas por hora')

    return f"Predicción: {int(prediction)} bicicletas alquiladas", fig

# Ejecutar la app
if __name__ == '__main__':
    app.run_server(debug=True)


"pip" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


ModuleNotFoundError: No module named 'dash'